In [1]:
import os
import cv2
import csv
from copy import deepcopy
from rembg import remove
import warnings

In [2]:
warnings.filterwarnings(action='ignore')

In [3]:
def get_box_coord(label, width, height):
    """
    x1 : min_x
    x2 : max_x
    y1 : min_y
    y2 : max_y
    """
    cl, x, y, w, h = label
    cl = int(cl)
    x = float(x) * width
    y = float(y) * height
    w = float(w) * width
    h = float(h) * height
    x1, x2 = round(x - w / 2), round(x + w / 2)
    y1, y2 = round(y - h / 2), round(y + h / 2)
    return x1, x2, y1, y2

In [4]:
def label_list(path):
    with open(path) as f:
        r = csv.reader(f, delimiter=' ')
        label_list = list(r)
    return label_list

In [5]:
img_list = os.listdir('./images')
for i in img_list:
    if i[-3:] != 'jpg':
        continue
    
    output_path = f"./images/{i}"
    
    im = cv2.imread(f"./images/{i}")
    height, width, _ = im.shape
    path1 = f'../labels/{i[:-4]}.txt'
    if not os.path.isfile(path1): # 라벨 없을떄
        print(f"{path1} no labels")
        im = remove(im)
        cv2.imwrite(output_path, im)
        continue
    label = label_list(path1)
    
    remove_im = remove(im)
    new_im = deepcopy(remove_im[:,:,:3])
    for j in label:
        x1, x2, y1, y2 = get_box_coord(j, width, height)
        origin_im = im[y1:y2, x1:x2]
        re = remove(im[y1:y2, x1:x2])
        total = (origin_im.shape[0] * origin_im.shape[1] * origin_im.shape[2])
        wt = (sum(sum(origin_im[:,:,0]==re[:,:,0])) + sum(sum(origin_im[:,:,1]==re[:,:,1])) + sum(sum(origin_im[:,:,2]==re[:,:,2])))
        if wt / total >= 0.55:
            new_im[y1:y2, x1:x2] = re[:,:,:3]
        else:
            new_im[y1:y2, x1:x2] = origin_im
    cv2.imwrite(output_path, new_im)            
        